In [504]:
import sys
import os
import pandas as pd
import random

sys.path.append(os.path.abspath('../src'))

from models import Player, Team, Individual
from utils import create_valid_team, create_valid_individual, generate_initial_population
from crossover import (
    team_swap_crossover,
    player_swap_crossover,
    uniform_team_crossover,
    position_based_crossover,
    gene_level_crossover
)
from mutation import (
    random_player_mutation,
    swap_players_between_teams,
    replace_team_mutation,
    expensive_player_mutation,
    cross_position_mutation,
    forward_reset_mutation,
    rotate_teams_mutation
)
from selection import (
    tournament_selection,
    roulette_wheel_selection,
    truncation_selection,
    rank_selection,
    stochastic_universal_sampling,
    elitism,
    select_parents
)

from main import main 

In [505]:
players_csv = pd.read_csv('../data/players.csv')

In [506]:
players = []
for idx, row in players_csv.iterrows():
    p = Player(
        player_id=idx + 1,
        name=row['Name'],
        position=row['Position'],
        skill=row['Skill'],
        cost=row['Salary (€M)']
    )

    players.append(p)

print(f"Número de jogadores carregados: {len(players)}")

Número de jogadores carregados: 35


In [507]:
used_ids = set()
team = create_valid_team(players, used_ids)

if team:
    print([str(p) for p in team.players])
    print("Válida:", team.is_valid())
    print("Custo total:", team.total_cost())
    print("Skill média:", team.average_skill())
else:
    print("⚠️ Não foi possível gerar uma equipa válida.")

['Jordan Smith (GK) - Skill: 88 | €M100', 'Ethan Howard (DEF) - Skill: 80 | €M70', 'Jaxon Griffin (DEF) - Skill: 79 | €M65', 'Nathan Wright (MID) - Skill: 92 | €M120', 'Austin Torres (MID) - Skill: 82 | €M80', 'Elijah Sanders (FWD) - Skill: 93 | €M140', 'Tyler Jenkins (FWD) - Skill: 80 | €M70']
Válida: True
Custo total: 645
Skill média: 84.85714285714286


In [508]:
ind = create_valid_individual(players)
if ind is not None:
    print("Indivíduo válido: ", ind.is_valid())
    print("Fitness (desvio padrão negativo):", ind.fitness())
else:
    print("⚠️ Não foi possível criar um indivíduo válido.")

Indivíduo válido:  True
Fitness (desvio padrão negativo): -0.7948815853282034


In [509]:
parent1 = create_valid_individual(players)
parent2 = create_valid_individual(players)

if not parent1 or not parent2:
    print("Erro: não foi possível gerar pais válidos.")
else:
    print("Pais criados com sucesso.")


Pais criados com sucesso.


In [510]:
if parent1 is None or parent2 is None:
    print("❌ Um dos pais é None — falha ao gerar indivíduo válido.")
else:
    child1, child2 = team_swap_crossover(parent1, parent2)
    print("Team Swap -> Fitness Child1:", child1.fitness(), "| Child2:", child2.fitness())
    print("Child1 válido?", child1.is_valid())
    print("Child2 válido?", child2.is_valid())

Team Swap -> Fitness Child1: -0.42378277069117926 | Child2: -1.4917459979664727
Child1 válido? True
Child2 válido? True


In [511]:
if parent1 is None or parent2 is None:
    print("❌ Um dos pais é None — falha ao gerar indivíduo válido.")
else:
    child1, child2 = player_swap_crossover(parent1, parent2)
    print("Player Swap -> Fitness Child1:", child1.fitness(), "| Child2:", child2.fitness())
    print("Child1 válido?", child1.is_valid())
    print("Child2 válido?", child2.is_valid())

Player Swap -> Fitness Child1: -1.059842113770938 | Child2: -9999
Child1 válido? True
Child2 válido? False


In [512]:
if parent1 is None or parent2 is None:
    print("❌ Um dos pais é None — falha ao gerar indivíduo válido.")
else:
    child = uniform_team_crossover(parent1, parent2)
    print("Uniform Crossover -> Fitness:", child.fitness())
    print("Válido:", child.is_valid())

Uniform Crossover -> Fitness: -1.396789321323466
Válido: True


In [513]:
if parent1 is None or parent2 is None:
    print("❌ Um dos pais é None — falha ao gerar indivíduo válido.")
else:
    child = position_based_crossover(parent1, parent2)
    print("Position-Based Crossover -> Fitness:", child.fitness())
    print("Válido:", child.is_valid())

Position-Based Crossover -> Fitness: -9999
Válido: False


In [514]:
if parent1 is None or parent2 is None:
    print("❌ Um dos pais é None — falha ao gerar indivíduo válido.")
else:
    child = gene_level_crossover(parent1, parent2)
    print("Gene-Level Crossover -> Fitness:", child.fitness())
    print("Válido:", child.is_valid())

Gene-Level Crossover -> Fitness: -1.3862296537952234
Válido: True


In [515]:
individual_mut = create_valid_individual(players)
if individual_mut is None:
    print("⚠️ Não foi possível gerar um indivíduo válido.")
else:
    print("Indivíduo base gerado com sucesso.")

Indivíduo base gerado com sucesso.


In [516]:
mutated = random_player_mutation(individual_mut, players)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.0356896548795327
Depois: -1.0356896548795327
Válido: True


In [517]:
mutated = swap_players_between_teams(individual_mut)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.0356896548795327
Depois: -1.182870945399112
Válido: True


In [518]:
mutated = replace_team_mutation(individual_mut, players)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.0356896548795327
Depois: -1.0356896548795327
Válido: True


In [519]:
mutated = expensive_player_mutation(individual_mut, players)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.0356896548795327
Depois: -1.0356896548795327
Válido: True


In [520]:
mutated = cross_position_mutation(individual_mut)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.0356896548795327
Depois: -1.2758350606723468
Válido: True


In [521]:
mutated = forward_reset_mutation(individual_mut, players)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.0356896548795327
Depois: -1.0356896548795327
Válido: True


In [522]:
mutated = rotate_teams_mutation(individual_mut)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.0356896548795327
Depois: -1.0356896548795327
Válido: True


In [523]:
population = generate_initial_population(10, players)
fitnesses = [ind.fitness() for ind in population]

In [524]:
parent = tournament_selection(population, fitnesses)
print("Torneio -> Fitness:", parent.fitness())

Torneio -> Fitness: -0.9536182439016362


In [525]:
parent = roulette_wheel_selection(population, fitnesses)
print("Roleta -> Fitness:", parent.fitness())

Roleta -> Fitness: -0.9536182439016362


In [526]:
parent = truncation_selection(population, fitnesses, top_percent=0.3)
print("Truncamento -> Fitness:", parent.fitness())

Truncamento -> Fitness: -0.7948815853282005


In [527]:
parent = stochastic_universal_sampling(population, fitnesses)
print("SUS -> Fitness:", parent.fitness())

SUS -> Fitness: -0.8778452283278404


In [528]:
elites = elitism(population, fitnesses, num_elites=2)
print("Elites -> Fitnesses:", [e.fitness() for e in elites])

Elites -> Fitnesses: [-0.4179639668093641, -0.7948815853282005]


In [529]:
p1, p2 = select_parents(population, fitnesses, method="tournament")
print("select_parents (Torneio) ->", p1.fitness(), p2.fitness())

select_parents (Torneio) -> -0.8916849448186975 -0.8916849448186975


In [530]:
best_ind, history = main(players, 
                         pop_size=20, 
                         generations=30, 
                         elite_count=2,
                         selection_method="tournament",
                         crossover_rate=0.9,
                         mutation_rate=0.8)

Geração 0 - Melhor fitness: 947.7838
Geração 1 - Melhor fitness: 972.0058
Geração 2 - Melhor fitness: 977.1429
Geração 3 - Melhor fitness: 977.1429
Geração 4 - Melhor fitness: 977.1429
Geração 5 - Melhor fitness: 981.9298
Geração 6 - Melhor fitness: 982.8571
Geração 7 - Melhor fitness: 982.8571
Geração 8 - Melhor fitness: 982.8571
Geração 9 - Melhor fitness: 982.8571
Geração 10 - Melhor fitness: 987.2225
Geração 11 - Melhor fitness: 987.2225
Geração 12 - Melhor fitness: 987.2225
Geração 13 - Melhor fitness: 987.2225
Geração 14 - Melhor fitness: 987.2225
Geração 15 - Melhor fitness: 989.3096
Geração 16 - Melhor fitness: 989.3096
Geração 17 - Melhor fitness: 989.3096
Geração 18 - Melhor fitness: 990.9649
Geração 19 - Melhor fitness: 990.9649
Geração 20 - Melhor fitness: 990.9649
Geração 21 - Melhor fitness: 994.2857
Geração 22 - Melhor fitness: 994.2857
Geração 23 - Melhor fitness: 1000.0000
Geração 24 - Melhor fitness: 1000.0000
Geração 25 - Melhor fitness: 1000.0000
Geração 26 - Melhor